# 다음의 News Classification Using Word2Vec

In [19]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
from numba import jit
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [2]:
print (device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1053707643325616978
]


In [3]:
cores = int(multiprocessing.cpu_count())

In [4]:
import Basic_Module as bm

## Load Data

In [23]:
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)
extDaumData = daumData.loc[:,['id','title','extracted_keywords']].copy()
print ('Daum : {}'.format(daumData.shape))

Daum : (9372, 11)


## Stopwords

In [6]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Document Labeling

In [46]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags category')

## Category

In [8]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


In [9]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'

In [10]:
daumData.head()

,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{지역, 연기, 공장, 남성들, 건강, 세보 효과, 부작용}"
1,5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{식후, 자세, 점심 식사, 입냄새, 건강, 철분, 낮잠, 디스크, 치아, 커피}"
2,5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{남편, 철원, 지인, 앙심, 범행, 주변, 진술, 경찰, 성관계, 아들}"
3,5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{검찰, 네이처리퍼블릭, 유통업체, 징역, 롯데, 혐의, 신영자 이사장, 매장}"
4,5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{남자친구, 범행, 경찰, 혐의, 유치장, 폭행, 과자, 쪽지}"


In [11]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Train data set으로 부터 TF-IDF Vectorizer을 만듦

In [12]:
trainName = './data/pre_data/news_train_test_Data/pre_data_word2vec_train_for_news_classification_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 12852/12852 [00:00<00:00, 1057773.49it/s]


(12852, 73416)
vocab size : 73416


## Word2Vec Model

### Twitter

#### News to Tagged Document

In [13]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_daum_news_by_ct_for_word2vec_news_classification.pickled'):
    daumData2 = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_ct_for_word2vec_news_classification.pickled', 'rb'))
else:
    daumData2 = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(daumData2, open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_ct_for_word2vec_news_classification.pickled', 'wb'))

#### Load Model

In [14]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-500_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-500_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-500_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### Model1

In [15]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('word2vec', model1,'ct')

 11%|█▏        | 9196/80534 [00:00<00:00, 91870.06it/s]

Word2Vec(vocab=80534, size=500, alpha=0.025)


100%|██████████| 80534/80534 [00:00<00:00, 99014.35it/s]
9it [00:00, 85.40it/s]

running time : 0:00:00.820691
Vectorizing Data


9372it [01:31, 102.61it/s]


scaling Data
total running time : 0:01:32.398697


In [16]:
classifierList = glob(classifierPath+'*'+modelName)

In [34]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

NeuralNetwork_2
XGBoost
RandomForestClassifier
LogisticRegression
NeuralNetwork_1
SVC


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 372008.45it/s]
9372it [00:00, 683492.44it/s]
9372it [00:00, 567131.48it/s]


CPU times: user 42.4 s, sys: 615 ms, total: 43 s
Wall time: 42.9 s


#### Model2

In [36]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('word2vec', model2,'ct')

  8%|▊         | 6817/80534 [00:00<00:01, 68132.70it/s]

Word2Vec(vocab=80534, size=500, alpha=0.025)


100%|██████████| 80534/80534 [00:00<00:00, 101052.03it/s]
2it [00:00, 11.58it/s]

running time : 0:00:00.799942
Vectorizing Data


9372it [01:16, 122.47it/s]


scaling Data
total running time : 0:01:17.543729


In [37]:
classifierList = glob(classifierPath+'*'+modelName)

In [38]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

NeuralNetwork_1
SVC
XGBoost
LogisticRegression
RandomForestClassifier
NeuralNetwork_2


In [39]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 595086.25it/s]
9372it [00:00, 753320.50it/s]
9372it [00:00, 381781.79it/s]


CPU times: user 34.9 s, sys: 677 ms, total: 35.5 s
Wall time: 36 s


#### Model3

In [40]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('word2vec', model3,'ct')

 14%|█▍        | 11307/80534 [00:00<00:00, 113030.22it/s]

Word2Vec(vocab=80534, size=500, alpha=0.025)


100%|██████████| 80534/80534 [00:00<00:00, 89427.06it/s] 
0it [00:00, ?it/s]

running time : 0:00:00.907499
Vectorizing Data


9372it [01:26, 108.00it/s]


scaling Data
total running time : 0:01:28.050622


In [41]:
classifierList = glob(classifierPath+'*'+modelName)

In [42]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

NeuralNetwork_1
SVC
XGBoost
RandomForestClassifier
LogisticRegression
NeuralNetwork_2


In [43]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 684384.93it/s]
9372it [00:00, 461584.73it/s]
9372it [00:00, 444425.79it/s]


CPU times: user 38 s, sys: 884 ms, total: 38.8 s
Wall time: 41.8 s


### Mecab

#### News to Tagged Document

In [47]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_daum_news_by_mecab_for_word2vec_news_classification.pickled'):
    daumData2 = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_mecab_for_word2vec_news_classification.pickled', 'rb'))
else:
    daumData2 = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(daumData2, open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_mecab_for_word2vec_news_classification.pickled', 'wb'))

100%|██████████| 9372/9372 [04:24<00:00, 35.44it/s]


#### Load Model

In [48]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-500_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-mecab.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-500_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-mecab.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-500_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-mecab.model')

#### Model1

In [49]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('word2vec', model1,'mecab')

  9%|▉         | 7594/80260 [00:00<00:00, 75917.09it/s]

Word2Vec(vocab=80260, size=500, alpha=0.025)


100%|██████████| 80260/80260 [00:00<00:00, 108698.58it/s]
26it [00:00, 257.61it/s]

running time : 0:00:00.742853
Vectorizing Data


9372it [00:24, 377.59it/s]


scaling Data
total running time : 0:00:25.734698


In [50]:
classifierList = glob(classifierPath+'*'+modelName)

In [51]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

RandomForestClassifier
XGBoost
NeuralNetwork_2
LogisticRegression
NeuralNetwork_1
SVC


In [52]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 725606.69it/s]
9372it [00:00, 747788.86it/s]
9372it [00:00, 721319.31it/s]


CPU times: user 42.4 s, sys: 541 ms, total: 42.9 s
Wall time: 42.3 s


#### Model2

In [53]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('word2vec', model2,'mecab')

 14%|█▍        | 11090/80260 [00:00<00:00, 110864.42it/s]

Word2Vec(vocab=80260, size=500, alpha=0.025)


100%|██████████| 80260/80260 [00:00<00:00, 107248.24it/s]
34it [00:00, 331.21it/s]

running time : 0:00:00.751867
Vectorizing Data


9372it [00:24, 378.75it/s]


scaling Data
total running time : 0:00:25.608120


In [54]:
classifierList = glob(classifierPath+'*'+modelName)

In [55]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

RandomForestClassifier
XGBoost
NeuralNetwork_2
LogisticRegression
SVC
NeuralNetwork_1


In [56]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 735283.98it/s]
9372it [00:00, 660644.65it/s]
9372it [00:00, 821024.63it/s]


CPU times: user 32.7 s, sys: 399 ms, total: 33.1 s
Wall time: 32.3 s


#### Model3

In [57]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('word2vec', model3,'mecab')

 15%|█▌        | 12330/80260 [00:00<00:00, 123259.27it/s]

Word2Vec(vocab=80260, size=500, alpha=0.025)


100%|██████████| 80260/80260 [00:00<00:00, 132960.45it/s]
80it [00:00, 382.88it/s]

running time : 0:00:00.606284
Vectorizing Data


9372it [00:24, 385.70it/s]


scaling Data
total running time : 0:00:25.017162


In [58]:
classifierList = glob(classifierPath+'*'+modelName)

In [59]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

NeuralNetwork_2
LogisticRegression
SVC
NeuralNetwork_1
RandomForestClassifier
XGBoost


In [60]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 430731.83it/s]
9372it [00:00, 768399.58it/s]
9372it [00:00, 704828.98it/s]


CPU times: user 36.7 s, sys: 645 ms, total: 37.4 s
Wall time: 37.5 s
